In [92]:
import numpy as np
import matplotlib.pyplot as plt

# 확률과정

- 시간이 흘러가면서 나오는 여러개의 결과들이 각각의 확률분포에 의해 결정되는 과정을 확률과정이라 함
- 수학적으로 보자면? $\to$ random variable들의 sequence(나열)가 확률과정이다.

$$
\begin{aligned}
\{X(t) : t \in T\} \text{ or } \{X_t\}
\end{aligned}
$$

- 예를 들자면?
    - 어제 비가 왔을때 다음날,다다음날,...,한달뒤에 비가 올 까?
    - 롤하는데 가장 최근에 한 판 이기고 다음판 다다음판 다다다음판은 이길 수 있을까?
- Note : 확률과정은 하나의 outcome(결과) $\omega$에 대해서 관찰된 뭔가이다.
    - 어떤 결과가 하나 먼저 나오고 그 다음에 쭈욱 확률에 의해 결정되는 확률변수들의 나열이다.
    - 하나의 $\omega$에 대응하는 확률과정 이렇게 많이 말함.

# 시뮬레이션 : 여러날을 구경해보자~

어제 비가 왔다면 $\to$ 오늘은 맑을 확률 0.7,오늘도 비 올 확률 0.3<br>
어제 맑았다면 $\to$ 오늘도 맑을 확률 0.4,오늘은 비 올 확률 0.3  이라 해보자.
<br>
확률변수 $X_t$를 t번째 날에 비가 올 확률이라고 하고 비가 오는 것을 1이라 하면 다음과 같은 식이 나온다.

$$
\begin{aligned}
P(X_t| X_{t-1} = 1) = 
\begin{cases}
0.7 \quad(X_t = 0) \\ 0.3 \quad(X_t = 1)
\end{cases}
\\
P(X_t| X_{t-1} = 0) = 
\begin{cases}
0.4 \quad(X_t = 0) \\ 0.6 \quad(X_t = 1)
\end{cases}

\end{aligned}
$$

이러한 법칙을 따르는 세상이 있다고 해보자.<br>


오늘 비가 왔을때, 10000일 동안 비가오는지 안오는지를 확률과정을 관측해보자.
1. 가장 최근에 날씨가 있다. 비가 왔다고 해보자. $\to w = 1$
2. 위의 확률법칙을 따라서 날씨가 관찰된다.
3. 1,2를 10000번 관찰하면 $\to$ 길이가 10000인 하나의 확률과정을 관측

In [67]:
def rain(before):
    """
    전날의 날씨에 따른 조건부 확률분포에 의해 다음날의 날씨를 결정해주는 함수
    input
    before : 전날의 날씨
    output 
    다음날의 날씨
    """
    if before == 1: #전날에 비가 왔다면
        return int(np.random.rand() < 0.3) #다음날에 비 올 확률은 확 깍여서 0.3
    if before == 0: #전날에 비가 안왔다면
        return int(np.random.rand() <0.6) #다음날에 비가 올 확률은 확 높아져서 0.6

In [71]:
def rain_process(today):
    result = [today]
    for i in range(10000):
        result.append(rain(result[-1])) #가장 최근날씨가 계속 input으로 들어감
    return result

In [74]:
sp1 = rain_process(today = 1)
len(sp1),sp1[:10]

(10001, [1, 0, 1, 0, 1, 1, 0, 1, 0, 0])

10000일동안의 확률과정을 한 번 관찰했다.(길이가 10001인 이유는 given된 outcome이 하나 있어서다.)

# 진짜 궁금한거 : 어떤날에 비가 올까요?

- 전날비가오면 다음날 비가오고 뭐 이런법칙을 따르는 확률과정을 안다고 해보자.
- 내가 정말로 알고싶은건? 알아내고 싶은 거는? $\to $오늘 비가 왔을때 10000일 시간이 흐른 뒤 비가 올 확률이다.
    - 이걸 알아낼 수 있다면? $\to$ 비가 오니까 밖에 안나가고 집에만 있을 수 있다.(원래도 잘 안나가긴 함.)
    - 더써보자면 ... 주식투자를 오늘 했을때 장기적으로 오를지 안오를지(!!)
    - 이 밖에 등등... 여러가지에서 미래를 가늠해보고 싶은 것들
- 좀 정리해보자면 이게 궁금한거다.

::: {.callout-note icon=false }
## 목적
확률과정 $\{X_t\}$가 존재할 때, 임의의 시점 $t$에서 $X_t$의 확률분포를 구하고자 한다.
:::

# 궁금한거 구해보기

- 궁금한거 : 오늘 비가 왔을 때 10000일 뒤에 비가 올 확률은?

$$P(X_{10000} = 1) = \text{ ?}$$

- 여기서는 몇 가지 방법을 소개함.
- 기본적인 방법들과 앞으로 확률과정론에서 배울 미래를 예측하는 여러가지 방법에 대한 소개

## 방법1: 확률과정 여러번 반복해보기(시뮬레이션을 통한 근사)

- 확률변수의 평균을 모르면 여러번 관측해서 표본평균으로 근사할 수 있다.
- 이 방법을 사용해서 해보자.


In [99]:
# 1. 여러개의 확률과정 만들기 (여기서는 첫날에 비가 왔다고 했을 때 10000일 동안을 관찰)
many_pcs = np.array([rain_process(1) for i in range(10000)])
# 2. X_1000에서 비가 왔는지 안왔는지 여러번 관측값 가져오기
days_10000 = many_pcs[:,-1]
# 비가 올 확률은?
print("비가 올 확률 :",np.mean(days_10000))

비가 올 확률 : 0.4621


- 근데 이 방법으로 문제를 모든 풀 수 있을까? : NO
    - 과거$\to$미래,과거$\to$미래, 진짜 여러번 과거로 돌아가서 관측해봐야 함.
    - 따라서 현실에서 불가능.

## 방법2 : 직접 확률 구해보기(이론적 계산)

- 여기서는 이론적으로 비가 올 확률을 구해보자.

**(비가오는 법칙,Refresh)**
$$
\begin{aligned}
P(X_t| X_{t-1} = 1) = 
\begin{cases}
0.7 \quad(X_t = 0) \\ 0.3 \quad(X_t = 1)
\end{cases}
\\
P(X_t| X_{t-1} = 0) = 
\begin{cases}
0.4 \quad(X_t = 0) \\ 0.6 \quad(X_t = 1)
\end{cases}

\end{aligned}
$$

**(이번에 비가 올 확률)**

\begin{aligned}
&P(X_t=1)\\
& = \text{이전에는 맑고 이번에는 비올 확률 + 이전에는 비가오고 이번에도 비올 확률 (sum rule)}\\
& = \text{이전에 맑을 확률 * 이전에 맑을때 지금은 비가 올 확률 + 이전에 비올 확률 * 이전에 비가오고 이번에도 비가 올 확률(product rule)}\\
&= P(X_{t-1} = 0 ) \times 0.6 + P(X_{t-1}=1) \times 0.3
\end{aligned}

- 맑을 확률도 계산해서 써보면...

**(이번에 맑을 확률)**

$$P(X_t=0) = P(X_{t-1} = 0 ) \times 0.4 + P(X_{t-1}=1) \times 0.7$$

- 행렬을 써서 표현해보자.

**(matrix form)**

\begin{aligned}
\begin{bmatrix}
P(X_t = 0) \\
P(X_t = 1) \\
\end{bmatrix} = 
\begin{bmatrix}
0.4 & 0.7 \\
0.6 & 0.3
\end{bmatrix}
\begin{bmatrix}
P(X_{t-1} = 0)\\
P(X_{t-1} = 1)
\end{bmatrix}
\end{aligned}


- 위 행렬을 다음과 같이 약속해보자.

$$
\begin{aligned}
&\mu_t = \bold P\mu_{t-1}\\
&\text{where }\mu_0 = \begin{bmatrix}0 \\ 1\end{bmatrix}
\end{aligned}
$$

- $\mu_t$는 임의의 시점 $t$에서의 확률변수 $X_t$의 확률분포이다.
- $P$는 상태간의 이동에 대한 확률이다.
    - 이전에 비가왔을때(이전상태) 다음날에 비가 올 확률(다음상태)과 같은 상태변화가 일어날 확률을 나타낸다.
- $\mu_0$는 outcome $\omega$에 대응하는 확률분포로 mass가 1 또는 0에 몰려있다고 하자.(유의)
- 비가 왔을 경우에 대응하는 10000일 동안의 확률과정이므로 outcome $\omega$에 대한 $\mu_0$는 다음과 같다.

**(첫 날에 비가 올까?)**

- 이는 $X_1$의 확률분포인 $\mu_1$을 구하면 알 수 있다.

$$
\begin{aligned}
&\boldsymbol{\mu}_1 = {\bf P}\boldsymbol{\mu}_{0}\\
&\text{where }\mu_0 = \begin{bmatrix}0 \\ 1\end{bmatrix}
\end{aligned}
$$

In [103]:
P = np.array([0.4,0.7,0.6,0.3]).reshape(2,2)
mu0 = np.array([0,1]).reshape(2,1)
P,mu0

(array([[0.4, 0.7],
        [0.6, 0.3]]),
 array([[0],
        [1]]))

In [104]:
mu1 = P @ mu0
mu1

array([[0.7],
       [0.3]])

**(둘째 날에 비가 올까?)**

- $X_2$의 확률분포를 구하는 문제다.

$$
\begin{aligned}
\boldsymbol{\mu}_2 &= {\bf P}\boldsymbol{\mu}_{1}\\
&= {\bf P}{\bf P}\boldsymbol{\mu}_{0}\\
&= {\bf P}^2\boldsymbol{\mu}_{0}\\


\end{aligned}
$$

In [106]:
P = np.array([0.4,0.7,0.6,0.3]).reshape(2,2)
mu0 = np.array([0,1]).reshape(2,1)
mu2 = P@P@mu0
mu2

array([[0.49],
       [0.51]])

**(n번째 날에 비가 오나?)**
- $X_n$의 확률분포를 구하는 문제다.

$$
\begin{aligned}
\boldsymbol{\mu}_n &= {\bf P}\boldsymbol{\mu}_{n-1}\\
&= {\bf P}^2\boldsymbol{\mu}_{n-2}\\
&= {\bf P}^3\boldsymbol{\mu}_{n-3}\\
&= \quad\quad \vdots \\
&= {\bf P}^n\boldsymbol{\mu}_{0}\\
\end{aligned}
$$

In [110]:
mu10000 = np.linalg.matrix_power(P,10000) @ mu0
mu10000

array([[0.53846154],
       [0.46153846]])

- 이러한 index가 1에 있는 값이 비가 올 확률이므로 읽어보면 !

In [111]:
print("비가 올 확률 :",np.mean(mu10000[1]))

비가 올 확률 : 0.4615384615383571


- 근사적으로 구한 값과 이론적으로 구한 값이 거의 비슷함을 알 수 있다..!